## Organize GeoMAC Fire Perimeter Data

Fire perimeter data for all US fires is available from the interagency fire coordination center known as "GeoMAC"

https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/historic_fire_data/us_historic_fire_perims_dd83_METADATA.html

Unfortunately, these datasets are split by time stamp, but do not have identical schema.  Some were evidently opened in ArcGIS, which automatically creates additional columns.  Others were not opened or not saved after opening, and have fewer columns.

If an identical schema was present, we could load and append all files directly from their remote URLs.  However, given the differences, we elected to open each in geopandas, and copy over into a common record set only the core data.  Also, date times were not provided in ISO format as required by OmniSci. Therefore we also cleaned those up and dropped any "NaT" (not a time) records.

In [97]:
import os, glob

In [98]:
%run ../omnigeo.ipynb

Using pymapd version 0.7.1
OmniSci connection established on python global variable "con"
Using CPU memory (/dev/shm) for temp geo files
df: /dev/shm/csvs: No such file or directory


### All GeoMAC US Fires 2010-2017 inclusive

In [58]:
all_fires = 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/historic_fire_data/US_HIST_FIRE_PERIMTRS_DD83.zip'
table_name = 'fire_perimeters_all_final_us_2010_2017'

In [27]:
q = f"COPY {table_name} FROM '{all_fires}' WITH (GEO='True');"
result = mapdql(q)    

Executing query: COPY fire_perimeters_all_final_us_2010_2017 FROM 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/historic_fire_data/US_HIST_FIRE_PERIMTRS_DD83.zip' WITH (GEO='True');


In [29]:
con.get_table_details(table_name)

[ColumnDetails(name='year_', type='STR', nullable=True, precision=0, scale=0, comp_param=32),
 ColumnDetails(name='acres', type='DOUBLE', nullable=True, precision=0, scale=0, comp_param=0),
 ColumnDetails(name='fire_name', type='STR', nullable=True, precision=0, scale=0, comp_param=32),
 ColumnDetails(name='unit_id', type='STR', nullable=True, precision=0, scale=0, comp_param=32),
 ColumnDetails(name='irwinid', type='STR', nullable=True, precision=0, scale=0, comp_param=32),
 ColumnDetails(name='fire_num', type='STR', nullable=True, precision=0, scale=0, comp_param=32),
 ColumnDetails(name='st_area_sh', type='DOUBLE', nullable=True, precision=0, scale=0, comp_param=0),
 ColumnDetails(name='st_length_', type='DOUBLE', nullable=True, precision=0, scale=0, comp_param=0),
 ColumnDetails(name='omnisci_geo', type='MULTIPOLYGON', nullable=True, precision=23, scale=4326, comp_param=32)]

In [83]:
q = "SELECT fire_name, year_ as fire_year, round(acres) as acres "
q += f'FROM {table_name} '
q += "GROUP BY fire_name, year_, acres "
q += "ORDER BY year_ DESC, acres DESC"
print(q)
df = con.select_ipc(q)
df

SELECT fire_name, year_ as fire_year, round(acres) as acres FROM fire_perimeters_all_final_us_2010_2017 GROUP BY fire_name, year_, acres ORDER BY year_ DESC, acres DESC


,fire_name,fire_year,acres
0,Jungo,2018,519.0
1,OKS - Starbuck,2017,662593.0
2,THOMAS,2017,281894.0
3,Bridge Coulee,2017,270723.0
4,WEST MIMS,2017,206743.0
5,Roosters Comb,2017,204879.0
6,Chetco Bar,2017,191125.0
7,Rice Ridge,2017,160187.0
8,Snowstorm,2017,154454.0
9,Meyers,2017,128462.0


Conclusion: missing most of the interesting fires of late 2018...

geomac_points = 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/historic_fire_data/2018_sit_rep_pts_dd83.zip'
table_name = 'fire_us_sit_rep_pts_2018'

In [100]:
if not table_name in con.get_tables():
    q = f'COPY {table_name} '
    q += f"FROM '{geomac_points}' " 
    q += "WITH (GEO='True');"
    result = mapdql(q)
else:
    print(f'Table {table_name} already exists on server, drop first to override')

Table fire_us_sit_rep_pts_2018 already exists on server, drop first to override


#### All US FIres Final Perimeters by Year from USGS GeoMAC

In [101]:
fire_year = '2018'

In [102]:
def get_final_perimeters(year=fire_year, override=True):
    table_name = f'fire_all_us_final_perims_{fire_year}'
    override = True
    if table_name in con.get_tables() and not override:
        print(f'{table_name} already loaded')
        print(f'Skipping {table_name} re-creation, set override if desired')
    else:
        if override:
            mapdql(f'DROP TABLE {table_name}')
        remote_shapefile_zip = f'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/{fire_year}_fire_data/current_year_all_states/{fire_year}_perimeters_dd83.zip'
        q = f"COPY {table_name} FROM '{remote_shapefile_zip}' WITH (GEO='True');"
        result = mapdql(q)    

In [103]:
get_final_perimeters()

Executing query: DROP TABLE fire_all_us_final_perims_2018
Executing query: COPY fire_all_us_final_perims_2018 FROM 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/2018_fire_data/current_year_all_states/2018_perimeters_dd83.zip' WITH (GEO='True');


In [ ]:
mapdql(f'SELECT * FROM {table_name}')
df = pd.DataFrame(result.fetchall())
df.head(3)

#### Scrape Camp Fire Web Page to Get All Perimeters

In [104]:
fire_name = 'Camp' # title case
fire_state = 'California' # full name, title case
fire_year = '2018' # as a 4 character string

Example URLs from GeoMAC as of Mar 2019

In [105]:
fire_page = 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/2018_fire_data/California/Camp/'
fire_page = 'https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/2018_fire_data/California/Woolsey/'

In [106]:
import requests
from bs4 import BeautifulSoup

In [107]:
def get_fire_perim_urls(year, state, firename):
    #expects 4 digit year, full state name and fire name in title case
    fire_page = f"https://rmgsc.cr.usgs.gov/outgoing/GeoMAC/{year}_fire_data/{state}/{firename}/"
    response = requests.get(fire_page)
    html = response.text
    soup = BeautifulSoup(html)
    urls = []
    for a in soup.find_all('a', href=True):
        if 'zip' in a['href']:
            fullurl = 'https://rmgsc.cr.usgs.gov/' + a['href']
            print ("Found the URL: \n", fullurl)
            urls.append(fullurl)
    print(f'Found {len(urls)} fire perimeters')
    return(urls)

In [108]:
camp_urls = get_fire_perim_urls(fire_year, fire_state, fire_name)

Found the URL: 
 https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181108_1754_dd83.zip
Found the URL: 
 https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181109_0000_dd83.zip
Found the URL: 
 https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181109_1902_dd83.zip
Found the URL: 
 https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181109_2313_dd83.zip
Found the URL: 
 https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181110_0104_dd83.zip
Found the URL: 
 https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181110_2110_dd83.zip
Found the URL: 
 https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181111_1848_dd83.zip
Found the URL: 
 https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181112_0031_dd83.zip
Found the URL: 
 https:/

Load fire perimeters into separate tables with unique names per time step, for later cleanup

In [109]:
def load_fire_perims(urls, overwrite=False):
    count = 0
    for url in urls:
        source_filename = url.split('/')[-1]
        state, fire_name, fire_date_yyyymmdd, fire_time_hhmm, fire_crs = source_filename.split('_')
        table_name = f'fire_{state}_{fire_name}_{fire_date_yyyymmdd}_{fire_time_hhmm}'
        print(f'Loading url {count} of {len(urls)} to OmniSci table: {table_name}')
        if not table_name in con.get_tables() or overwrite:
            mapdql(f"DROP TABLE {table_name}")
            q = f"COPY {table_name} FROM '{url}' WITH (GEO='True');"
            try:
                result = mapdql(q)
                print(f'Added date {fire_date_yyyymmdd} time {fire_time_hhmm}')
            except:
                print(f'Problem loading url {url}')
        else:
            print(f'Table {table_name} already exists, overwrite if needed')

In [110]:
load_fire_perims(camp_urls)

Loading url to OmniSci table: fire_ca_camp_20181108_1754
Executing query: DROP TABLE fire_ca_camp_20181108_1754
Executing query: COPY fire_ca_camp_20181108_1754 FROM 'https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181108_1754_dd83.zip' WITH (GEO='True');
Added date 20181108 time 1754
Loading url to OmniSci table: fire_ca_camp_20181109_0000
Executing query: DROP TABLE fire_ca_camp_20181109_0000
Executing query: COPY fire_ca_camp_20181109_0000 FROM 'https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181109_0000_dd83.zip' WITH (GEO='True');
Added date 20181109 time 0000
Loading url to OmniSci table: fire_ca_camp_20181109_1902
Executing query: DROP TABLE fire_ca_camp_20181109_1902
Executing query: COPY fire_ca_camp_20181109_1902 FROM 'https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181109_1902_dd83.zip' WITH (GEO='True');
Added date 20181109 time 1902
Loading url to OmniSci table: fire

Added date 20181120 time 2005
Loading url to OmniSci table: fire_ca_camp_20181121_2105
Executing query: DROP TABLE fire_ca_camp_20181121_2105
Executing query: COPY fire_ca_camp_20181121_2105 FROM 'https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181121_2105_dd83.zip' WITH (GEO='True');
Added date 20181121 time 2105
Loading url to OmniSci table: fire_ca_camp_20181125_1822
Executing query: DROP TABLE fire_ca_camp_20181125_1822
Executing query: COPY fire_ca_camp_20181125_1822 FROM 'https://rmgsc.cr.usgs.gov//outgoing/GeoMAC/2018_fire_data/California/Camp/ca_camp_20181125_1822_dd83.zip' WITH (GEO='True');
Added date 20181125 time 1822


In [ ]:
omnitables_df[omnitables_df['name'].str.contains(fire_name)]

#### Camp Fire Specific Perimeters with Time Stamps

Schemas vary by addition of OBJECTID, SHAPE_Area fields which I think can probably be safely dropped (since auto-added by ESRI along the way).

But cannot import into one table until schema are identical.  So make a first pass, putting into separate tables.  Then we can ALTER those tables and concatentate them

In [ ]:
process_perimeters('fire_woolsey_perimeters','woolsey')

In [ ]:
# refresh tables df
omnitables_df = pd.DataFrame(con.get_tables(), columns={'name'}).sort_values('name')

In [ ]:
def list_fire_perims(fire):
    l = []
    for index, row in omnitables_df.iterrows():
        if fire in row['name'].lower():
            print(row['name'])
            l.append(row['name'])
    return(l)

In [ ]:
list_fire_perims('woolsey_perimeter_')

argh, this doesn't work because Omni doesn't support dropping columns.  I guess must CTAS with the 'good' columns instead into new version tables - what a pain!

### Copy good ones...

In [111]:
def good_col_names():
    cols_tuples = con.get_table_details('fire_ca_camp_20181110_0104')
    cols_df = pd.DataFrame(cols_tuples)
    all_col_names = cols_df['name'].unique()
    good_col_names = []
    for name in all_col_names:
        if name != 'OBJECTID' and name != 'SHAPE_Area':
            good_col_names.append(name)
    return(good_col_names)

In [112]:
good_col_names()

['irwinid',
 'mapmethod',
 'unitIDProt',
 'unitIDOwn',
 'incidentID',
 'fireName',
 'perDatTime',
 'comments',
 'agency',
 'active',
 'latest',
 'compParID',
 'fireYear',
 'dateCrnt',
 'inciwebId',
 'firecode',
 'mergeid',
 'compfirecd',
 'fireNum',
 'ComplexNm',
 'state',
 'inComplex',
 'GISACRES',
 'omnisci_geo']

In [ ]:
def standard_ddl(col_names):
    cols_tuples = con.get_table_details('fire_ca_camp_20181110_0104')
    cols_df = pd.DataFrame(cols_tuples)
    ddl = '('
    for index, row in cols_df.iterrows():
        name = row['name']
        if name in col_names:
            ddl += '{} {}, '.format(name, row['type'])
    ddl = ddl[:-2]
    ddl += ')'
    return(ddl)

In [ ]:
standard_ddl(good_col_names())

Get rid of Field Names accidentally added by ArcGIS.  Could in other cases into SHSPE_Perimeter

In [ ]:
good_col_names()

In [ ]:
def onlythegoodstuff(table_name_selector):
    fire_tables = omnitables_df[omnitables_df['name'].str.contains(table_name_selector)]
    for index, row in fire_tables.iterrows():
        orig_table = row['name']
        print('Working on table {}'.format(orig_table))
        good_table = orig_table + '_v2'
        if not good_table in con.get_tables():
            ss = 'SELECT '
            for name in good_col_names():
                ss += '{}, '.format(name)
            ss = ss[:-2]
            ss += ' FROM {}'.format(orig_table)
            query = 'CREATE TABLE {} AS ({})'.format(good_table, ss)
            try:
                mapdql(query)
            except:
                print('Problem creating standardized version of table {}'.format(orig_table))
        

In [ ]:
mapdql('drop table fire_perim_woolsey')

In [ ]:
onlythegoodstuff('woolsey_perimeter_')

In [ ]:
def append_perims(firename):
    # refresh tables df
    omnitables_df = pd.DataFrame(con.get_tables(), columns={'name'}).sort_values('name')
    df_list = []

    for index, row in omnitables_df.iterrows():
        name = row['name'].lower()
        if f'fire_{firename}_perimeter_' in name and '_v2' in name:
            print(name)
            try:
                result = mapdql('SELECT * FROM {}'.format(name))
                df_list.append(pd.DataFrame(result.fetchall()))
            except:
                print('failed to select')

    if len(df_list) > 0:
        df2 = pd.concat(df_list)
    else:
        print('Data frame list empty')
    df2.columns = good_col_names()
    droplist = {'irwinid','unitIDProt','unitIDOwn','incidentID','fireName','agency','mergeid','compfirecd','fireNum','ComplexNm','inComplex','state','dateCrnt','compParID','inciwebId', 'firecode', 'fireYear'}
    final_cols = list(set(good_col_names()) - set(droplist))
    df2.drop(droplist,inplace=True, axis=1)
    return(df2)

In [ ]:
df2 = append_perims('woolsey')

In [ ]:
df2.head()

In [ ]:
# perDatTime imported as a string, but is actually a date/time

Make a mega dataframe then fix perDatTime in the df and save back?

In [ ]:
ddl = standard_ddl(final_cols)
ddl

In [ ]:
# over-ride perDatTime STR
# (YYYY-MM-DD HH:MM:SS)
# ? will this handle ours with am/pm and not military 24 hour clock?
ddl = ddl.replace('perDatTime STR','perDatTime TIMESTAMP')
ddl = ddl.replace('STR','TEXT')
ddl

In [ ]:
def create_final_table(df2, firename):
    table_name = f'fire_{firename}_perimeters'
    query = "CREATE TABLE {} {};".format(table_name, ddl)
    mapdql(query)
    con.load_table_rowwise(table_name, df2.itertuples(index=False, name=None))

In [ ]:
create_final_table(df2, 'woolsey')

In [ ]:
cwd = os.cwd()

In [ ]:
!pwd

In [ ]:
query = "COPY (SELECT * FROM fire_woolsey_perimeters) to '/home/mapdadmin/demo/fire/fire_woolsey_perimeters.csv'"
mapdql(query)

In [ ]:
!head fire_woolsey_perimeters.csv